# LLM API — Getting Started

This notebook shows how to call the **GPT-OSS** model hosted on `hub.qazcode.ai`.

The server uses the **OpenAI-compatible API**, so you can use the `openai` Python library or plain `requests`.

## 1. Install dependencies

In [12]:
#!pip install openai

## 2. Configuration

In [9]:
API_KEY = "YOUR_API_KEY"  # replace with your key
HUB_URL = "URL"

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url=HUB_URL,
    api_key=API_KEY,  # replace with your key
)

MODEL = "oss-120b"

## 3. Basic Chat Completion

In [5]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Hello! Who are you?"}
    ],
)

print(response.choices[0].message.content)

Hello! I’m ChatGPT, an AI language model created by OpenAI. I’m here to help answer questions, brainstorm ideas, explain concepts, or just chat—whatever you need. How can I assist you today?


## 4. System Prompt + User Message

In [6]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a medical diagnosis assistant. Given patient symptoms, suggest the most probable diagnosis with an ICD-10 code."
        },
        {
            "role": "user",
            "content": "Patient presents with fever, dry cough, and shortness of breath lasting 5 days."
        }
    ],
)

print(response.choices[0].message.content)

**Probable Diagnosis:**  
**COVID‑19 (acute respiratory infection caused by SARS‑CoV‑2)**  

**ICD‑10‑CM Code:** **U07.1** – *COVID‑19, virus identified*  

**Rationale**  
- **Fever + dry cough + dyspnea** are the classic triad for COVID‑19.  
- The symptom duration of **~5 days** fits the typical early phase of SARS‑CoV‑2 infection (symptoms usually appear 2–14 days after exposure).  
- In the current epidemiologic context, COVID‑19 remains the most common cause of an acute febrile respiratory illness with a dry cough and shortness of breath.  

**Key Differentials to Keep in Mind**  
| Condition | ICD‑10‑CM | Typical distinguishing features |
|-----------|----------|-----------------------------------|
| Influenza, virus identified | J10.1 | Sudden onset, myalgias, chills; often accompanied by productive cough |
| Viral pneumonia, unspecified | J12.9 | May present with sputum production, infiltrates on chest imaging |
| Bacterial pneumonia (e.g., pneumococcal) | J13 | Usually produc

## 5. Structured JSON Output

In [7]:
import json

SYSTEM_PROMPT = """You are a clinical decision support system.
Given patient symptoms, return a JSON object:
{
  "diagnoses": [
    {"rank": 1, "icd_code": "...", "name": "...", "explanation": "..."},
    ...
  ]
}
Return top 3 diagnoses ranked by likelihood. Respond with JSON only."""

symptoms = "Severe headache, nausea, vomiting, neck stiffness, sensitivity to light."

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": symptoms},
    ],
)

result = json.loads(response.choices[0].message.content)
for d in result["diagnoses"]:
    print(f"[{d['rank']}] {d['icd_code']} — {d['name']}")
    print(f"    {d['explanation']}\n")

[1] G00.9 — Bacterial meningitis, unspecified
    The combination of acute severe headache, nausea, vomiting, neck stiffness and photophobia strongly suggests meningeal irritation. Bacterial meningitis is a medical emergency and is the most common cause of this symptom cluster in adults presenting with rapid onset.

[2] I60.9 — Nontraumatic subarachnoid hemorrhage, unspecified
    A sudden, severe (“thunderclap”) headache with associated nausea, vomiting, neck stiffness and photophobia can also be caused by bleeding into the subarachnoid space. Early imaging is required to rule this out.

[3] G03.0 — Viral meningitis
    Viral meningitis presents similarly to bacterial meningitis but is generally less fulminant. The same meningeal signs are present, making it a reasonable differential when infectious work‑up is pending.



In [ ]:
import requests

response = requests.post(
    "https://{}/chat/completions".format(HUB_URL),
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    },
    json={
        "model": "oss-120b",
        "messages": [{"role": "user", "content": "Hello!"}],
    },
)

print(response.json()["choices"][0]["message"]["content"])